# Imports and configs

In [1]:
!pip install -q autogluon.tabular ray==2.10.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.1/65.1 MB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 352.2/352.2 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.2/266.2 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.1/64.1 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.2/68.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 81.1 MB/s eta 0:00:00


In [2]:
from autogluon.tabular import TabularPredictor
import pandas as pd
import numpy as np
import shutil

In [3]:
class CFG:
    train_path = '/kaggle/input/ai-competition-africa/train.csv'
    test_path = '/kaggle/input/ai-competition-africa/test.csv'
    sample_sub_path = '/kaggle/input/ai-competition-africa/sample_submission.csv'
    
    target = 'quality'
    time_limit = 3600 * 11

# Loading data

In [4]:
train = pd.read_csv(CFG.train_path, index_col='id')
test = pd.read_csv(CFG.test_path, index_col='id')

train[CFG.target] = train[CFG.target].astype(int) - 3

# Fitting the predictor

In [5]:
predictor = TabularPredictor(
    problem_type='multiclass',
    eval_metric='quadratic_kappa',
    label=CFG.target,
    verbosity=2
)

No path specified. Models will be saved in: "AutogluonModels/ag-20241224_221506"


In [6]:
predictor.fit(
    train_data=train,
    time_limit=CFG.time_limit,
    excluded_model_types = ["KNN"],
    presets='best_quality'
)

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Nov 10 10:07:59 UTC 2024
CPU Count:          4
Memory Avail:       30.30 GB / 31.35 GB (96.7%)
Disk Space Avail:   19.50 GB / 19.52 GB (99.9%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets of the data. Then holdout validation data is used to detect stacked 

(_ray_fit pid=2863) [1000]	valid_set's multi_logloss: 1.16174	valid_set's quadratic_kappa: 0.364966


(_dystack pid=156) 	0.3767	 = Validation score   (quadratic_kappa)
(_dystack pid=156) 	61.91s	 = Training   runtime
(_dystack pid=156) 	5.85s	 = Validation runtime
(_dystack pid=156) Fitting model: NeuralNetFastAI_r191_BAG_L1 ... Training model for up to 6006.90s of the 9306.86s of remaining time.
(_dystack pid=156) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (4 workers, per: cpus=1, gpus=0, memory=0.05%)
(_ray_fit pid=2983) Metric quadratic_kappa is not supported by this model - using log_loss instead
(_ray_fit pid=2985) No improvement since epoch 12: early stopping
(_ray_fit pid=2984) Metric quadratic_kappa is not supported by this model - using log_loss instead [repeated 3x across cluster]
(_ray_fit pid=2986) No improvement since epoch 12: early stopping [repeated 2x across cluster]
(_ray_fit pid=3118) Metric quadratic_kappa is not supported by this model - using log_loss instead
(_ray_fit pid=3115) Metric quadratic_kappa is not supported by

(_ray_fit pid=3558) [1000]	valid_set's multi_logloss: 1.05245	valid_set's quadratic_kappa: 0.393563
(_ray_fit pid=3560) [1000]	valid_set's multi_logloss: 1.07425	valid_set's quadratic_kappa: 0.367485
(_ray_fit pid=3697) [1000]	valid_set's multi_logloss: 1.06009	valid_set's quadratic_kappa: 0.365449
(_ray_fit pid=3755) [1000]	valid_set's multi_logloss: 1.0655	valid_set's quadratic_kappa: 0.374883


(_dystack pid=156) 	0.3783	 = Validation score   (quadratic_kappa)
(_dystack pid=156) 	68.45s	 = Training   runtime
(_dystack pid=156) 	8.48s	 = Validation runtime
(_dystack pid=156) Fitting model: NeuralNetTorch_r22_BAG_L1 ... Training model for up to 5664.77s of the 8964.73s of remaining time.
(_dystack pid=156) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (4 workers, per: cpus=1, gpus=0, memory=0.02%)
(_dystack pid=156) 	0.3899	 = Validation score   (quadratic_kappa)
(_dystack pid=156) 	117.86s	 = Training   runtime
(_dystack pid=156) 	0.4s	 = Validation runtime
(_dystack pid=156) Fitting model: XGBoost_r33_BAG_L1 ... Training model for up to 5543.48s of the 8843.44s of remaining time.
(_dystack pid=156) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (4 workers, per: cpus=1, gpus=0, memory=0.15%)
(_dystack pid=156) 	0.3638	 = Validation score   (quadratic_kappa)
(_dystack pid=156) 	77.64s	 = Training   r

(_ray_fit pid=9458) [1000]	valid_set's multi_logloss: 1.07453	valid_set's quadratic_kappa: 0.354565
(_ray_fit pid=9457) [1000]	valid_set's multi_logloss: 1.06198	valid_set's quadratic_kappa: 0.359435
(_ray_fit pid=9456) [2000]	valid_set's multi_logloss: 1.05689	valid_set's quadratic_kappa: 0.397199 [repeated 2x across cluster]
(_ray_fit pid=9568) [1000]	valid_set's multi_logloss: 1.06759	valid_set's quadratic_kappa: 0.362371
(_ray_fit pid=9568) [2000]	valid_set's multi_logloss: 1.06907	valid_set's quadratic_kappa: 0.370364
(_ray_fit pid=9596) [1000]	valid_set's multi_logloss: 1.06649	valid_set's quadratic_kappa: 0.348576
(_ray_fit pid=9623) [1000]	valid_set's multi_logloss: 1.07136	valid_set's quadratic_kappa: 0.366452
(_ray_fit pid=9652) [1000]	valid_set's multi_logloss: 1.06889	valid_set's quadratic_kappa: 0.344332
(_ray_fit pid=9596) [2000]	valid_set's multi_logloss: 1.0656	valid_set's quadratic_kappa: 0.36255
(_ray_fit pid=9596) [3000]	valid_set's multi_logloss: 1.07042	valid_set's

(_dystack pid=156) 	0.369	 = Validation score   (quadratic_kappa)
(_dystack pid=156) 	172.16s	 = Training   runtime
(_dystack pid=156) 	51.43s	 = Validation runtime
(_dystack pid=156) Fitting model: RandomForest_r39_BAG_L1 ... Training model for up to 3764.61s of the 7064.58s of remaining time.
(_dystack pid=156) 	0.362	 = Validation score   (quadratic_kappa)
(_dystack pid=156) 	9.68s	 = Training   runtime
(_dystack pid=156) 	0.65s	 = Validation runtime
(_dystack pid=156) Fitting model: CatBoost_r167_BAG_L1 ... Training model for up to 3753.92s of the 7053.88s of remaining time.
(_dystack pid=156) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (4 workers, per: cpus=1, gpus=0, memory=0.06%)
(_dystack pid=156) 	0.3766	 = Validation score   (quadratic_kappa)
(_dystack pid=156) 	41.96s	 = Training   runtime
(_dystack pid=156) 	0.06s	 = Validation runtime
(_dystack pid=156) Fitting model: NeuralNetFastAI_r95_BAG_L1 ... Training model for up to 3708.49s

In [7]:
predictor.leaderboard(silent=True).style.background_gradient(subset=['score_val'], cmap='RdYlGn')

,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L3,0.422053,quadratic_kappa,51.402763,3506.425931,0.005152,4.581437,3,True,155
1,NeuralNetTorch_r30_BAG_L2,0.417835,quadratic_kappa,48.414383,3187.350746,2.019579,126.447066,2,True,137
2,NeuralNetTorch_r79_BAG_L2,0.414315,quadratic_kappa,48.835272,3146.517212,2.440468,85.613532,2,True,122
3,NeuralNetTorch_r86_BAG_L2,0.414081,quadratic_kappa,48.353113,3164.639723,1.958309,103.736043,2,True,139
4,WeightedEnsemble_L2,0.413674,quadratic_kappa,2.260907,726.749427,0.005646,4.372307,2,True,109
5,NeuralNetTorch_r158_BAG_L1,0.413609,quadratic_kappa,1.880407,693.459236,1.880407,693.459236,1,True,49
6,NeuralNetTorch_r22_BAG_L2,0.412998,quadratic_kappa,48.478505,3173.861677,2.083700,112.957997,2,True,127
7,NeuralNetTorch_BAG_L2,0.410677,quadratic_kappa,48.316121,3139.434535,1.921316,78.530855,2,True,119
8,NeuralNetTorch_r87_BAG_L1,0.405722,quadratic_kappa,0.323500,176.590209,0.323500,176.590209,1,True,70
9,NeuralNetTorch_r30_BAG_L1,0.404429,quadratic_kappa,0.370886,202.517098,0.370886,202.517098,1,True,28


# Creating a submission file

In [8]:
test_pred_probs = predictor.predict_proba(test)
sub = pd.read_csv(CFG.sample_sub_path)
sub[CFG.target] = np.argmax(test_pred_probs, axis=1)
sub[CFG.target] = sub[CFG.target] + 3
sub.to_csv('sub_autogluon.csv', index=False)
sub.head()

,id,quality
0,15000,5
1,15001,5
2,15002,6
3,15003,6
4,15004,5


In [9]:
shutil.rmtree("AutogluonModels")